In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [10]:
code = pd.read_excel(r"/content/NNDL.xlsx")
code

,Question,Correct_Code,Code_with_Error,Type_of_Error,Number_of_Loops,Header_and_Main_Declaration,Incomprehensible_Code,Comprehensible_Code_with_logical_errors,Comprehensible_code_with_syntax_errors,Correct_code_and_output,Total_marks
0,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() \n\ndouble n1...,"1c,1e",0,2,2.0,2,0,0,6
1,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,"#include <stdio.h>\n\nint main\n\ndouble n1, n...","1c,1e,1d",0,0,2.0,2,0,0,4
2,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() {\n\ndouble n...,"1b,1e",0,2,2.0,0,1,0,5
3,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() {\n\ndouble n...,1e,0,2,2.0,2,1,0,7
4,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() {\n\ndouble n...,2b,0,2,2.0,0,2,0,6
...,...,...,...,...,...,...,...,...,...,...,...
114,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint main(...,2a,0,2,2.0,1,2,2,9
115,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint mian(...,"1a,1c",0,2,2.0,2,0,0,6
116,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint main(...,3a,0,2,2.0,2,2,2,10
117,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint min()...,"1a,1e,2a",0,1,1.0,1,0,0,3


In [11]:
# Merge columns
merged_data = code['Question'] + ' ' + code['Code_with_Error']
merged_data

0      Largest among three numbers #include <stdio.h>...
1      Largest among three numbers #include <stdio.h>...
2      Largest among three numbers #include <stdio.h>...
3      Largest among three numbers #include <stdio.h>...
4      Largest among three numbers #include <stdio.h>...
                             ...                        
114    Calculate the roots of an equation #include<st...
115    Calculate the roots of an equation #include<st...
116    Calculate the roots of an equation #include<st...
117    Calculate the roots of an equation #include<st...
118    Calculate the roots of an equation #include<st...
Length: 119, dtype: object

In [12]:
# Fill missing values with empty strings
merged_data = merged_data.fillna('')

In [13]:
# Initialize and fit the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
vectorized_data = vectorizer.fit_transform(merged_data).toarray()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(vectorized_data, code['Total_marks'], test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [27]:
# Define the LSTM model
model = Sequential([
    Embedding(len(vectorizer.vocabulary_), 100, input_length=X_train.shape[1]),
    LSTM(units=128, return_sequences=True),
    Dropout(0.2),
    LSTM(units=128),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

In [28]:
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

In [29]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, callbacks=[early_stopping])


Epoch 1/50
3/3 [==============================] - 7s 854ms/step - loss: 50.9345 - val_loss: 46.3042
Epoch 2/50
3/3 [==============================] - 1s 405ms/step - loss: 47.6042 - val_loss: 35.6219
Epoch 3/50
3/3 [==============================] - 1s 389ms/step - loss: 31.7405 - val_loss: 11.0280
Epoch 4/50
3/3 [==============================] - 2s 685ms/step - loss: 9.6197 - val_loss: 7.5734
Epoch 5/50
3/3 [==============================] - 2s 661ms/step - loss: 5.7210 - val_loss: 8.9227
Epoch 6/50
3/3 [==============================] - 1s 436ms/step - loss: 6.8796 - val_loss: 7.9366
Epoch 7/50
3/3 [==============================] - 1s 419ms/step - loss: 5.7932 - val_loss: 7.4824
Epoch 8/50
3/3 [==============================] - 1s 404ms/step - loss: 5.7537 - val_loss: 7.4632
Epoch 9/50
3/3 [==============================] - 1s 423ms/step - loss: 5.9409 - val_loss: 7.4976
Epoch 10/50
3/3 [==============================] - 1s 428ms/step - loss: 5.2755 - val_loss: 7.5252
Epoch 11/50
3

In [30]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


1/1 [==============================] - 0s 162ms/step


In [31]:
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)


In [32]:
# Print the evaluation metrics
print(f"Train RMSE: {train_rmse:.2f}")
print(f"Train MAE: {train_mae:.2f}")
print(f"Train MAPE: {train_mape:.2f}")
print(f"Train R2: {train_r2:.2f}")

print(f"Test RMSE: {test_rmse:.2f}")
print(f"Test MAE: {test_mae:.2f}")
print(f"Test MAPE: {test_mape:.2f}")
print(f"Test R2: {test_r2:.2f}")

Train RMSE: 2.39
Train MAE: 1.98
Train MAPE: 0.46
Train R2: -0.03
Test RMSE: 2.31
Test MAE: 1.98
Test MAPE: 0.41
Test R2: -0.10
